In [1]:
pip install torch_geometric

Note: you may need to restart the kernel to use updated packages.


In [1]:
import json
import collections
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch_geometric.data import Data
from torch_geometric.utils.convert import to_networkx
from torch_geometric.nn import GCNConv

import networkx as nx


In [2]:
# Load data
train_path = "C:/Users/scorp/Downloads/train.csv"
test_path = "C:/Users/scorp/Downloads/test.csv"
train_data = pd.read_csv(train_path).fillna("")
test_data = pd.read_csv(test_path).fillna("")

In [3]:
train_data.head(5)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [10]:
train_data['input_text'] = train_data['title'].fillna('') + ' ' + train_data['author'].fillna('') + ' ' + train_data['text'].fillna('')
test_data['input_text'] = test_data['title'].fillna('') + ' ' + test_data['author'].fillna('') + ' ' + test_data['text'].fillna('')

In [11]:
train_data = train_data[:10000]
Y = train_data['label'].values

test_data = test_data

In [12]:
train_data.shape

(10000, 6)

In [13]:
test_data.shape

(5200, 5)

In [14]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx
import torch
from torch_geometric.data import Data

# Load your dataset
data = train_data


# Create a TF-IDF vectorizer and compute TF-IDF matrix
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(data['input_text'])

# Compute cosine similarity matrix
cosine_similarities = cosine_similarity(tfidf_matrix)

# Threshold for cosine similarity to determine edge connections
threshold = 0.7

# Create an undirected graph based on cosine similarity
G = nx.Graph()

# Add edges based on cosine similarity
for i in range(len(data)):
    for j in range(i + 1, len(data)):
        if cosine_similarities[i, j] > threshold:
            G.add_edge(i, j)

# Renumber the nodes to be within the expected interval [0, 499]
node_mapping = {node: i for i, node in enumerate(G.nodes())}
G = nx.relabel_nodes(G, node_mapping)
G_test = nx.relabel_nodes(G, node_mapping)

# Convert the graph to edge_index
edge_index_data = nx.to_edgelist(G)
edge_index = torch.tensor([(edge[0], edge[1]) for edge in edge_index_data], dtype=torch.long).t()

print(edge_index)


# Working for test data


# Create a TF-IDF vectorizer and compute TF-IDF matrix
vectorizer_test = TfidfVectorizer(stop_words='english')
tfidf_matrix_test = vectorizer_test.fit_transform(test_data['input_text'])

# Compute cosine similarity matrix
cosine_similarities_test = cosine_similarity(tfidf_matrix)

# Threshold for cosine similarity to determine edge connections
threshold = 0.7

# Add edges based on cosine similarity
for i in range(len(test_data)):
    for j in range(i + 1, len(test_data)):
        if cosine_similarities_test[i, j] > threshold:
            G_test.add_edge(i, j)

# Renumber the nodes to be within the expected interval [0, 499]
node_mapping_test = {node: i for i, node in enumerate(G_test.nodes())}
G_test = nx.relabel_nodes(G_test, node_mapping_test)

# Convert the graph to edge_index
edge_index_data_test = nx.to_edgelist(G_test)
edge_index_test = torch.tensor([(edge[0], edge[1]) for edge in edge_index_data_test], dtype=torch.long).t()

print(edge_index)

print("printing edge nodes for test")

print(edge_index_test)


tensor([[  0,   2,   4,  ..., 879, 882, 884],
        [  1,   3,   5,  ..., 880, 883, 885]])
tensor([[  0,   2,   4,  ..., 879, 882, 884],
        [  1,   3,   5,  ..., 880, 883, 885]])
printing edge nodes for test
tensor([[   0,    0,    0,  ..., 1119, 1120, 1123],
        [   1,    7,    8,  ..., 1121, 1121, 1124]])


In [15]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# Load your dataset
data = train_data

vocabulary = vectorizer.vocabulary_

# Create a TF-IDF vectorizer for text features (e.g., title and text)
vectorizer = TfidfVectorizer(vocabulary=vocabulary, stop_words='english')
text_features = vectorizer.fit_transform(data['title'] + ' ' + data['text'])

# Encode categorical features (e.g., author) using one-hot encoding
categorical_features = pd.get_dummies(data['author'])

# Combine text and categorical features
node_features = pd.concat([pd.DataFrame(text_features.toarray()), categorical_features], axis=1)
node_features_tensor = torch.tensor(node_features.values, dtype=torch.float32)

print(node_features_tensor)


# Code for test data
# Create a TF-IDF vectorizer for text features (e.g., title and text)

text_features_test = vectorizer.fit_transform(test_data['title'] + ' ' + test_data['text'])

# Encode categorical features (e.g., author) using one-hot encoding
categorical_features_test = pd.get_dummies(test_data['author'])

# Combine text and categorical features
node_features_test = pd.concat([pd.DataFrame(text_features_test.toarray()), categorical_features_test], axis=1)
node_features_tensor_test = torch.tensor(node_features_test.values, dtype=torch.float32)

print(node_features_tensor_test)




tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0503, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]])
tensor([[0.0000, 0.0780, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0184, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]])


In [27]:
num_features = node_features_tensor.shape[1]
num_features

124722

In [28]:
num_features_test = node_features_tensor_test.shape[1]
num_features_test

123773

In [29]:
import torch
from torch_geometric.data import Data, DataLoader

# Assuming you have node features, edge index, and labels
node_features = node_features_tensor
edge_index = edge_index
labels = Y

# Create a PyTorch Geometric Data object
data = Data(x=node_features, edge_index=edge_index, y=labels)

# Define batch size
batch_size = 32

# Create a DataLoader for batching
loader = DataLoader([data], batch_size=batch_size, shuffle=True)


# Code for test data

# Load the submit data for comparison
submit_path = "C:/Users/scorp/Downloads/submit.csv"
submit_data = pd.read_csv(submit_path)
submit_data = submit_data[:10000]

y_test = submit_data['label'].values

# Assuming you have node features, edge index, and labels
node_features_test = node_features_tensor_test
edge_index_test = edge_index_test

# Create a PyTorch Geometric Data object
data_test = Data(x=node_features_test, edge_index=edge_index_test, y=y_test)

# Define batch size
batch_size = 32

# Create a DataLoader for batching
test_loader = DataLoader([data_test], batch_size=batch_size, shuffle=True)



C:\Users\scorp\anaconda3\lib\site-packages\torch_geometric\deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)
C:\Users\scorp\anaconda3\lib\site-packages\torch_geometric\deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [31]:
from torch_geometric.nn import global_max_pool as gmp
from torch_geometric.nn import GATConv
from torch.nn import Linear

class GNN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        
        # Graph Convolutions
        self.conv1 = GATConv(in_channels, hidden_channels)
        self.conv2 = GATConv(hidden_channels, hidden_channels)
        self.conv3 = GATConv(hidden_channels, hidden_channels)

        # Readout
        self.lin_news = Linear(in_channels, hidden_channels)
        self.lin0 = Linear(hidden_channels, hidden_channels)
        self.lin1 = Linear(2*hidden_channels, out_channels)

    def forward(self, x, edge_index, batch):
        # Graph Convolutions
        h = self.conv1(x, edge_index).relu()
        h = self.conv2(h, edge_index).relu()
        h = self.conv3(h, edge_index).relu()

        # Pooling
        h = gmp(h, batch)

        # Readout
        h = self.lin0(h).relu()

        # According to UPFD paper: Include raw word2vec embeddings of news 
        # This is done per graph in the batch
        root = (batch[1:] - batch[:-1]).nonzero(as_tuple=False).view(-1)
        root = torch.cat([root.new_zeros(1), root + 1], dim=0)
        # root is e.g. [   0,   14,   94,  171,  230,  302, ... ]
        news = x[root]
        news = self.lin_news(news).relu()

        out = self.lin1(torch.cat([h, news], dim=-1))
        return torch.sigmoid(out)


import torch
import torch.optim as optim
import torch.nn.functional as F

# Initialize the model
model = GNN(num_features, 128, 1)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Define optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.BCELoss()  # Binary Cross-Entropy Loss for binary classification

# Training loop
model.train()  # Set the model to training mode

# Assuming you have already created the DataLoader 'loader'
count = 0
for epoch in range(10):    
    model.train()  # Set the model to training mode
    for data in loader:
        print("Count " + str(count))
        label_num = len(data.y[0])
        data = data.to(device)  # Move data to the device
        
        optimizer.zero_grad()  # Zero the gradients
        
        output = model(data.x, data.edge_index, data.batch)  # Forward pass
        
        print(output.shape)
        print("$$$$$")
        print(data.y[0][count])
        
        target = torch.tensor(data.y[0][count], dtype=torch.float32).view(-1, 1).to(device)
        print(target.shape)
        
        loss = criterion(output, target)  # Calculate loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update weights
        
        print("%%%%%")
    count = count + 1
        
    print(f'Epoch [{epoch+1}/{10}], Loss: {loss.item()}')

print("Training finished.")


Count 0
torch.Size([1, 1])
$$$$$
1
torch.Size([1, 1])
%%%%%
Epoch [1/10], Loss: 0.7052026987075806
Count 1
torch.Size([1, 1])
$$$$$
0
torch.Size([1, 1])
%%%%%
Epoch [2/10], Loss: 0.701367199420929
Count 2
torch.Size([1, 1])
$$$$$
1
torch.Size([1, 1])
%%%%%
Epoch [3/10], Loss: 0.6935752034187317
Count 3
torch.Size([1, 1])
$$$$$
1
torch.Size([1, 1])
%%%%%
Epoch [4/10], Loss: 0.686258852481842
Count 4
torch.Size([1, 1])
$$$$$
1
torch.Size([1, 1])
%%%%%
Epoch [5/10], Loss: 0.6740383505821228
Count 5
torch.Size([1, 1])
$$$$$
0
torch.Size([1, 1])
%%%%%
Epoch [6/10], Loss: 0.7284423112869263
Count 6
torch.Size([1, 1])
$$$$$
1
torch.Size([1, 1])
%%%%%
Epoch [7/10], Loss: 0.6556512713432312
Count 7
torch.Size([1, 1])
$$$$$
0
torch.Size([1, 1])
%%%%%
Epoch [8/10], Loss: 0.7417843341827393
Count 8
torch.Size([1, 1])
$$$$$
0
torch.Size([1, 1])
%%%%%
Epoch [9/10], Loss: 0.7423872351646423
Count 9
torch.Size([1, 1])
$$$$$
0
torch.Size([1, 1])
%%%%%
Epoch [10/10], Loss: 0.7372264862060547
Training fi

In [16]:
# Set the model to evaluation mode
# model = GNN(num_features_test, 128, 1)  # num_test_features is the number of features in your test data

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model.to(device)
model.conv1 = GATConv(num_features_test, 128)
model.lin_news = Linear(num_features_test, 128)
model.eval()

# Lists to store predictions and true labels
predictions = []
true_labels = []



# Iterate through the testing data
for data in test_loader:  # Assuming you have a DataLoader for the testing data
    
    data = data.to(device)  # Move data to the device
    
    with torch.no_grad():
        output = model(data.x, data.edge_index, data.batch)  # Forward pass
        
    # Convert the predicted probabilities to binary predictions (0 or 1)
    predicted_labels = (output >= 0.5).int()
    
    predictions.append(predicted_labels.cpu().numpy())  # Convert to numpy array
    true_labels.append(np.array(data.y))  # Convert to numpy array

# Convert the lists of predictions and true labels to numpy arrays
predictions = np.concatenate(predictions, axis=0)
true_labels = np.concatenate(true_labels, axis=0)

# Calculate accuracy
accuracy = (predictions == true_labels).mean()

print(f'Test Accuracy: {accuracy:.4f}')


Test Accuracy: 0.6320
